In [1]:
# !pip install --upgrade scipy pandas

In [2]:
import scipy.io
import pandas as pd

In [3]:
final_df = pd.DataFrame()

In [4]:
import os

directory_path = '../'

# Walk through all files and subdirectories in the specified directory
for root, dirs, files in os.walk(directory_path):
    for file_name in files:
        # Full path of the file
        file_path = os.path.join(root, file_name)

        if file_path.endswith('.mat') and 'idle' not in file_path and 'MAC' not in file_path:
            try:
                mat_data = scipy.io.loadmat(file_path, mat_dtype=True)
                cols = [i for i in range(1, 1601)]
                df = pd.DataFrame(mat_data['data'], columns = cols)
                df['label'] = 1
                final_df = pd.concat([final_df, df], ignore_index=True)
            except:
                print('File: ' + file_path + ' not processed!')

In [5]:
import os

directory_path = '../idle'

# Walk through all files and subdirectories in the specified directory
for root, dirs, files in os.walk(directory_path):
    for file_name in files:
        # Full path of the file
        file_path = os.path.join(root, file_name)

        if file_path.endswith('.mat'):
            try:
                mat_data = scipy.io.loadmat(file_path, mat_dtype=True)
                cols = [i for i in range(1, 1601)]
                df = pd.DataFrame(mat_data['data'], columns = cols)
                df['label'] = 0
                final_df = pd.concat([final_df, df], ignore_index=True)
            except:
                print('File: ' + file_path + ' not processed!')

In [6]:
final_df

,1,2,3,4,5,6,7,8,9,10,...,1592,1593,1594,1595,1596,1597,1598,1599,1600,label
0,-91,-91,-91,-91,-91,-91,-91,-91,-91,-92,...,-91,-91,-91,-91,-90,-91,-91,-91,-92,1
1,-104,-104,-86,-104,-104,-104,-104,-103,-104,-104,...,-104,-104,-79,-104,-104,-104,-104,-103,-104,1
2,-99,-99,-99,-99,-99,-99,-99,-100,-99,-99,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,1
3,-102,-102,-102,-102,-103,-102,-102,-102,-102,-102,...,-102,-102,-103,-102,-102,-102,-102,-102,-102,1
4,-95,-95,-95,-95,-95,-95,-95,-95,-95,-95,...,-93,-93,-92,-95,-95,-95,-95,-95,-95,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,-99,-99,-98,-99,-99,-99,-99,-99,-98,-99,...,-99,-99,-98,-98,-98,-98,-98,-99,-99,0
7996,-75,-104,-104,-104,-104,-104,-104,-103,-104,-104,...,-104,-104,-104,-104,-104,-104,-103,-104,-104,0
7997,-98,-97,-98,-98,-98,-98,-97,-97,-98,-98,...,-97,-98,-98,-98,-98,-98,-98,-98,-98,0
7998,-91,-91,-91,-91,-91,-91,-91,-91,-91,-91,...,-91,-91,-91,-91,-91,-91,-91,-91,-91,0


In [7]:
# !pip install scikit-learn
# !pip install imbalanced-learn

In [8]:
final_df['label'].value_counts()

label
1    6400
0    1600
Name: count, dtype: int64

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.tree import export_text
from imblearn.over_sampling import RandomOverSampler

target_column = 'label'

X = final_df.drop(target_column, axis=1)
y = final_df[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

In [10]:
d_tree = DecisionTreeClassifier()

d_tree.fit(X_train_resampled, y_train_resampled)

y_pred = d_tree.predict(X_test)

accuracy = metrics.accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.76


In [11]:
tree_rules = export_text(d_tree, feature_names=list(X.columns))
print(tree_rules)

|--- 177 <= -89.50
|   |--- 1191 <= -92.50
|   |   |--- 263 <= -106.50
|   |   |   |--- class: 0
|   |   |--- 263 >  -106.50
|   |   |   |--- 275 <= -101.50
|   |   |   |   |--- 1050 <= -102.50
|   |   |   |   |   |--- 1412 <= -102.50
|   |   |   |   |   |   |--- 546 <= -103.50
|   |   |   |   |   |   |   |--- 1023 <= -106.50
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |--- 1023 >  -106.50
|   |   |   |   |   |   |   |   |--- 599 <= -103.50
|   |   |   |   |   |   |   |   |   |--- 466 <= -104.50
|   |   |   |   |   |   |   |   |   |   |--- 587 <= -104.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 5
|   |   |   |   |   |   |   |   |   |   |--- 587 >  -104.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 3
|   |   |   |   |   |   |   |   |   |--- 466 >  -104.50
|   |   |   |   |   |   |   |   |   |   |--- 467 <= -103.50
|   |   |   |   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |

In [12]:
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.model_selection import train_test_split

target_column = 'label'

X = final_df.drop(target_column, axis=1)
y = final_df[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [13]:
d_tree = DecisionTreeClassifier()

d_tree.fit(X_train_resampled, y_train_resampled)

y_pred = d_tree.predict(X_test)

accuracy = metrics.accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.71


In [14]:
tree_rules = export_text(d_tree, feature_names=list(X.columns))
print(tree_rules)

|--- 177 <= -89.50
|   |--- 1556 <= -91.50
|   |   |--- 263 <= -106.50
|   |   |   |--- class: 0
|   |   |--- 263 >  -106.50
|   |   |   |--- 843 <= -101.50
|   |   |   |   |--- 974 <= -101.50
|   |   |   |   |   |--- 1598 <= -102.50
|   |   |   |   |   |   |--- 1370 <= -103.50
|   |   |   |   |   |   |   |--- 10 <= -106.50
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |--- 10 >  -106.50
|   |   |   |   |   |   |   |   |--- 52 <= -103.50
|   |   |   |   |   |   |   |   |   |--- 627 <= -106.50
|   |   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |   |   |--- 627 >  -106.50
|   |   |   |   |   |   |   |   |   |   |--- 487 <= -102.00
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 11
|   |   |   |   |   |   |   |   |   |   |--- 487 >  -102.00
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 2
|   |   |   |   |   |   |   |   |--- 52 >  -103.50
|   |   |   |   |   |   |   |   |   |-